In [ ]:

import numpy as np
import librosa
import soundfile as sf
from pathlib import Path
from tqdm import tqdm

# =============================================================================
# QUICK CONFIGURATION - UPDATE THESE
# =============================================================================

INPUT_FOLDER = Path('EdgeImpulse_Dataset/fold_10_test/training')
OUTPUT_FOLDER = Path('EdgeImpulse_Dataset/realistic_quick')
CREATE_BACKGROUND_CLASS = True  # Set to True to add background_noise class

# NEW: Support for organized folder structure
# Set to True if your files are organized like:
# training/air_conditioner/*.wav
# training/car_horn/*.wav
USE_SUBFOLDER_STRUCTURE = True

TARGET_SR = 16000
TARGET_LENGTH = 4.0

OUTPUT_FOLDER.mkdir(exist_ok=True, parents=True)

# =============================================================================
# SIMPLE NOISE FUNCTIONS
# =============================================================================

def add_realistic_noise(audio, sr=16000, noise_level='medium'):
    """
    Add realistic noise in one function
    noise_level: 'light', 'medium', 'heavy'
    """
    
    if noise_level == 'light':
        snr_db = np.random.uniform(20, 30)  # Clean
    elif noise_level == 'medium':
        snr_db = np.random.uniform(10, 20)  # Realistic
    else:  # heavy
        snr_db = np.random.uniform(5, 15)   # Challenging
    
    # Generate pink noise (most realistic)
    num_samples = len(audio)
    white = np.random.randn(num_samples)
    
    # Create 1/f (pink) spectrum
    fft = np.fft.rfft(white)
    freqs = np.fft.rfftfreq(num_samples, 1/sr)
    pink_filter = 1 / np.sqrt(freqs + 1)
    pink_fft = fft * pink_filter
    pink_noise = np.fft.irfft(pink_fft, n=num_samples)
    
    # Normalize noise
    pink_noise = pink_noise / np.max(np.abs(pink_noise))
    
    # Calculate noise power for target SNR
    signal_power = np.mean(audio ** 2)
    snr_linear = 10 ** (snr_db / 10)
    noise_power = signal_power / snr_linear
    
    # Scale noise
    pink_noise = pink_noise * np.sqrt(noise_power / np.mean(pink_noise ** 2))
    
    # Add to signal
    noisy = audio + pink_noise
    
    # Simulate distance (random)
    if np.random.rand() > 0.5:
        distance_factor = np.random.uniform(1.0, 2.0)
        cutoff_freq = 8000 / distance_factor
        
        fft = np.fft.rfft(noisy)
        freqs = np.fft.rfftfreq(len(noisy), 1/sr)
        filter_response = 1 / (1 + (freqs / cutoff_freq) ** 4)
        fft_filtered = fft * filter_response
        noisy = np.fft.irfft(fft_filtered, n=len(noisy))
        noisy = noisy * (1 / distance_factor ** 0.5)
    
    return noisy / np.max(np.abs(noisy) + 1e-10)

def create_background_noise_sample(duration=4.0, sr=16000):
    """
    Create one background noise sample
    """
    num_samples = int(duration * sr)
    
    # Mix of different noise types
    # 1. Low-frequency rumble (traffic)
    traffic = np.random.randn(num_samples)
    fft = np.fft.rfft(traffic)
    freqs = np.fft.rfftfreq(num_samples, 1/sr)
    filter_resp = 1 / (1 + (freqs / 600) ** 4)
    traffic = np.fft.irfft(fft * filter_resp, n=num_samples)
    
    # 2. Mid-frequency ambience
    ambience = np.random.randn(num_samples)
    fft = np.fft.rfft(ambience)
    filter_resp = 1 / (1 + (freqs / 2000) ** 2)
    ambience = np.fft.irfft(fft * filter_resp, n=num_samples)
    
    # 3. High-frequency hiss
    hiss = np.random.randn(num_samples) * 0.3
    
    # Mix with random weights
    w1, w2, w3 = np.random.dirichlet([1, 1, 1])
    background = w1 * traffic + w2 * ambience + w3 * hiss
    
    # Random amplitude modulation
    t = np.linspace(0, duration, num_samples)
    modulation = np.sin(2 * np.pi * np.random.uniform(0.1, 0.5) * t)
    modulation = (modulation + 1) / 2 + 0.5  # 0.5 to 1.5
    
    background = background * modulation
    
    return background / np.max(np.abs(background) + 1e-10)

# =============================================================================
# MAIN PROCESSING
# =============================================================================

def quick_augment(input_folder, output_folder, create_background=True):
    """
    Quick augmentation: Create 3 versions of each sample
    1. Original (clean)
    2. Light noise
    3. Medium noise
    
    Supports both flat and subfolder structures
    """
    
    print("="*60)
    print("QUICK REALISTIC AUGMENTATION")
    print("="*60)
    
    # Detect folder structure and collect files
    audio_files = []
    
    if USE_SUBFOLDER_STRUCTURE:
        print(f"\nUsing subfolder structure mode")
        print(f"Looking for class subfolders in: {input_folder}")
        
        # Find all class subfolders
        class_folders = [f for f in input_folder.iterdir() if f.is_dir()]
        
        if len(class_folders) == 0:
            print(f"\n❌ No subfolders found in {input_folder}")
            print("Set USE_SUBFOLDER_STRUCTURE = False if files are in root folder")
            return None
        
        print(f"\nFound {len(class_folders)} class folders:")
        
        for class_folder in sorted(class_folders):
            class_name = class_folder.name
            class_files = list(class_folder.glob('*.wav'))
            
            if len(class_files) > 0:
                print(f"  ✓ {class_name}: {len(class_files)} files")
                # Store files with their class name
                for f in class_files:
                    audio_files.append((f, class_name))
            else:
                print(f"  ⚠ {class_name}: No .wav files found")
    else:
        print(f"\nUsing flat folder structure mode")
        print(f"Looking for files in: {input_folder}")
        
        # Get all WAV files from root folder
        wav_files = list(input_folder.glob('*.wav'))
        
        if len(wav_files) == 0:
            print(f"\n❌ No .wav files found in {input_folder}")
            print("Set USE_SUBFOLDER_STRUCTURE = True if files are in subfolders")
            return None
        
        print(f"Found {len(wav_files)} files")
        
        # Extract class from filename (e.g., "air_conditioner.xxx.wav")
        for f in wav_files:
            class_name = f.name.split('.')[0]
            audio_files.append((f, class_name))
    
    print(f"\nTotal files to process: {len(audio_files)}")
    print(f"Creating 3 versions each: clean, light noise, medium noise")
    
    if create_background:
        print("Also creating background_noise class")
    
    stats = {'original': 0, 'light': 0, 'medium': 0, 'background': 0}
    class_counts = {}
    
    # Process each file
    print("\nProcessing...")
    for audio_file, class_name in tqdm(audio_files):
        try:
            audio, sr = librosa.load(audio_file, sr=TARGET_SR, mono=True)
            
            # Track class counts
            if class_name not in class_counts:
                class_counts[class_name] = 0
            class_counts[class_name] += 1
            
            # Ensure correct length
            target_samples = int(TARGET_LENGTH * TARGET_SR)
            if len(audio) != target_samples:
                if len(audio) > target_samples:
                    start = (len(audio) - target_samples) // 2
                    audio = audio[start:start + target_samples]
                else:
                    audio = np.pad(audio, (0, target_samples - len(audio)))
            
            # 1. Save original (clean)
            clean_file = output_folder / f"{class_name}.clean.{audio_file.stem}.wav"
            sf.write(clean_file, audio, TARGET_SR)
            stats['original'] += 1
            
            # 2. Light noise version
            light_noisy = add_realistic_noise(audio, TARGET_SR, noise_level='light')
            light_file = output_folder / f"{class_name}.light.{audio_file.stem}.wav"
            sf.write(light_file, light_noisy, TARGET_SR)
            stats['light'] += 1
            
            # 3. Medium noise version
            medium_noisy = add_realistic_noise(audio, TARGET_SR, noise_level='medium')
            medium_file = output_folder / f"{class_name}.medium.{audio_file.stem}.wav"
            sf.write(medium_file, medium_noisy, TARGET_SR)
            stats['medium'] += 1
            
        except Exception as e:
            print(f"\nError: {audio_file.name}: {e}")
    
    # Create background noise class if requested
    if create_background:
        print("\nCreating background_noise class...")
        
        # Create ~100 background samples
        num_bg_samples = 100
        
        for i in tqdm(range(num_bg_samples)):
            bg_sample = create_background_noise_sample(TARGET_LENGTH, TARGET_SR)
            
            bg_file = output_folder / f"background_noise.sample_{i:04d}.wav"
            sf.write(bg_file, bg_sample, TARGET_SR)
            stats['background'] += 1
    
    # Summary
    print("\n" + "="*60)
    print("COMPLETE!")
    print("="*60)
    print(f"\n✓ Original (clean): {stats['original']}")
    print(f"✓ Light noise: {stats['light']}")
    print(f"✓ Medium noise: {stats['medium']}")
    if create_background:
        print(f"✓ Background noise: {stats['background']}")
    
    total = sum(stats.values())
    print(f"\n✓ Total files: {total}")
    
    # Show per-class breakdown
    print("\n--- Per-Class Summary ---")
    for class_name, count in sorted(class_counts.items()):
        augmented_count = count * 3  # clean + light + medium
        print(f"  {class_name}: {count} → {augmented_count} samples")
    
    print(f"\n✓ Saved to: {output_folder}")
    
    
    return stats

# =============================================================================
# RUN
# =============================================================================

if __name__ == "__main__":
    
    stats = quick_augment(INPUT_FOLDER, OUTPUT_FOLDER, CREATE_BACKGROUND_CLASS)
